In [12]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [13]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
assets = [:USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net    │
├────────────┼────────────┼──────────────┼─────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │   0.0246253 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │   0.0560778 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │   0.0302388 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │   0.0125024 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │  -0.0201209 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │ -0.00119203 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │     0.01818 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │  -0.0566908 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │   -0.042229 │
│

In [14]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))

417×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2022-09-30
┌────────────┬─────────────┬──────────────┬─────────────┐
│            │ USA_Net     │ World_ex_USA │ EM_Net      │
├────────────┼─────────────┼──────────────┼─────────────┤
│ 1988-01-31 │   0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │   0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │  -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │  0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │  0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │   0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │  -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │   -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │      ⋮      │      ⋮       │      ⋮      │
│ 2022-03-31 │     0.03417 │    0.0115373 │  -0.0228443 │
│ 2022-04-30 │  -0.0952756 │   -0.0679247 │  -0.0572341 │
│ 2022-05-31 │ -0.00270982 │    0.0082682 │   0.0043937 │
│ 2022-06-30 │  -0.0868359 │   -0.0988188 │  -0.068761

In [15]:
[assets; :ACWI_Net]

4-element Vector{Symbol}:
 :USA_Net
 :World_ex_USA
 :EM_Net
 :ACWI_Net

In [34]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1507,1.246,0.0676,0.4027,-2.257,-0.84,-0.2075,0.0703,1.6368,0.4485,0.1412
World_ex_USA,0.1653,1.4555,0.0499,0.4126,-2.7986,-0.913,-0.2363,0.1497,2.4191,0.3017,0.1038
EM_Net,0.233,1.893,0.0684,0.5702,-4.1,-1.0973,-0.3213,0.1423,3.896,0.2936,0.1038


In [45]:
hmm = hmm_est(period_returns, 2)
regime_summary(hmm[1], assets, 36)


Means
Standard deviations
Regimes transition matrix


Regime,USA_Net,World_ex_USA,EM_Net
1,-0.235,-0.323,-0.306
2,0.468,0.436,0.515


Regime,USA_Net,World_ex_USA,EM_Net
1,0.356,0.375,0.542
2,0.165,0.199,0.275


"Correlations in regime 1"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.884,0.794
World_ex_USA,0.884,1.000,0.845
EM_Net,0.794,0.845,1.000


"Correlations in regime 2"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.805,0.653
World_ex_USA,0.805,1.000,0.823
EM_Net,0.653,0.823,1.000


Regime,1,2
1,0.928,0.072
2,0.038,0.962


In [46]:
n_assets = length(assets)
scenarios = simulate_hmm(hmm[1], n_assets, 180, 100_000)

for a in 1:n_assets
    print_percentiles(scenarios[a,:,:], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12, string(assets[a]))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.4453,-0.3847,-0.3655,-0.3529,-0.3524,-0.3554,-0.3578,-0.352,-0.3584,-0.3529,-0.3522,-0.3546,-0.3569,-0.3564,-0.3544
0.05,-0.3563,-0.2849,-0.2638,-0.2533,-0.251,-0.2536,-0.2538,-0.2517,-0.2548,-0.2531,-0.2488,-0.2504,-0.2543,-0.2522,-0.2535
0.25,-0.1404,-0.0507,-0.023,-0.0152,-0.0151,-0.0162,-0.0143,-0.0139,-0.0149,-0.013,-0.0131,-0.0135,-0.0143,-0.0132,-0.0136
0.5,-0.0003,0.0812,0.0972,0.1004,0.1011,0.1014,0.1013,0.1017,0.1002,0.1019,0.1018,0.102,0.1015,0.1017,0.1025
0.75,0.1258,0.18,0.1888,0.1896,0.19,0.1897,0.1893,0.1901,0.1899,0.1899,0.1904,0.1906,0.1905,0.1894,0.1906
0.95,0.2856,0.303,0.3058,0.3057,0.3057,0.3045,0.3046,0.3071,0.3036,0.305,0.3054,0.3064,0.3056,0.305,0.3068
0.98,0.3542,0.3552,0.3562,0.3554,0.3558,0.356,0.3536,0.3559,0.3539,0.3545,0.3548,0.3567,0.3566,0.356,0.357


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.493,-0.4302,-0.4072,-0.3967,-0.3991,-0.4024,-0.4033,-0.3991,-0.4007,-0.3981,-0.3971,-0.4014,-0.4014,-0.3987,-0.3982
0.05,-0.399,-0.3279,-0.3026,-0.2934,-0.2914,-0.2952,-0.2969,-0.2917,-0.2948,-0.2941,-0.2892,-0.2899,-0.2945,-0.2929,-0.2918
0.25,-0.1741,-0.0821,-0.0536,-0.047,-0.0453,-0.0468,-0.046,-0.0457,-0.0465,-0.0448,-0.0455,-0.045,-0.046,-0.0443,-0.0448
0.5,-0.0261,0.0597,0.0775,0.0815,0.0814,0.0813,0.0816,0.0828,0.0826,0.0824,0.0821,0.0819,0.0831,0.0826,0.0827
0.75,0.1097,0.1722,0.1829,0.184,0.1854,0.1847,0.184,0.1861,0.1853,0.1855,0.1862,0.1858,0.1853,0.1845,0.186
0.95,0.2862,0.3148,0.319,0.3196,0.3192,0.3208,0.3193,0.3203,0.3179,0.3191,0.3205,0.3203,0.3199,0.32,0.3219
0.98,0.3599,0.3742,0.3732,0.3769,0.3748,0.3762,0.3745,0.3773,0.3737,0.3755,0.3771,0.3765,0.3751,0.3757,0.379


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.6646,-0.5742,-0.5448,-0.5318,-0.5283,-0.5378,-0.5338,-0.5359,-0.5368,-0.5299,-0.5309,-0.5319,-0.5379,-0.5318,-0.5295
0.05,-0.5274,-0.4321,-0.3942,-0.3828,-0.3806,-0.388,-0.3885,-0.3839,-0.3879,-0.3799,-0.3794,-0.3803,-0.3863,-0.3814,-0.385
0.25,-0.2122,-0.1012,-0.069,-0.0605,-0.0606,-0.0622,-0.061,-0.0578,-0.0607,-0.06,-0.0594,-0.0569,-0.0595,-0.0593,-0.0601
0.5,-0.01,0.0804,0.1,0.1029,0.103,0.103,0.103,0.1053,0.1035,0.1049,0.1033,0.1045,0.1047,0.1045,0.1056
0.75,0.1755,0.2297,0.2416,0.2432,0.2431,0.2431,0.2428,0.2442,0.243,0.2419,0.2431,0.2438,0.2433,0.2444,0.2444
0.95,0.4278,0.4328,0.4335,0.4318,0.4319,0.4354,0.4315,0.4362,0.4304,0.4328,0.4329,0.4343,0.4345,0.4354,0.4334
0.98,0.5372,0.5204,0.5198,0.5165,0.5168,0.518,0.5168,0.5193,0.5153,0.517,0.5163,0.5177,0.5176,0.5173,0.5173


In [ ]:
y_data = annualise(scenarios[1, :, 61:73],12)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

"mean"

0.07390805936756287

0.1013562411057124

"std"

0.169924210958029

"skewness"

-0.8005888840836739

"kurtosis"

0.9418571728582332

In [48]:
weights = [0.321, 0.67, 0.009]
#weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(acwi, [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0.02,-0.4632,-0.4044,-0.382,-0.3753,-0.374,-0.3814,-0.3796,-0.3772,-0.3768,-0.3744,-0.3723,-0.3736,-0.3761,-0.3776
0.05,-0.3687,-0.3027,-0.2828,-0.273,-0.2717,-0.2755,-0.275,-0.2724,-0.2753,-0.2708,-0.2678,-0.2692,-0.2737,-0.2708
0.25,-0.1496,-0.0647,-0.039,-0.0318,-0.0332,-0.0334,-0.0316,-0.0314,-0.0322,-0.0296,-0.0307,-0.0295,-0.0308,-0.0294
0.5,-0.0045,0.0696,0.0853,0.0878,0.0888,0.089,0.0897,0.0897,0.0893,0.0907,0.0904,0.0905,0.0913,0.0899
0.75,0.1237,0.1737,0.1824,0.1837,0.1846,0.1848,0.1837,0.1853,0.1849,0.1853,0.1851,0.185,0.1851,0.1849
0.95,0.2844,0.3051,0.3077,0.3087,0.3078,0.3084,0.3067,0.3094,0.306,0.3066,0.3077,0.3076,0.3071,0.306
0.98,0.3522,0.3599,0.3598,0.3595,0.3593,0.3602,0.3584,0.3619,0.3576,0.3586,0.3574,0.36,0.3595,0.3583


In [56]:
display("mean")
display(mean(annualise(acwi[:,60:72],12)))
display("std")
display(std(annualise(acwi[:,60:72],12)))
display("skewness")
display(skewness(annualise(acwi[:,60:72],12)))
display("kurtosis")
display(kurtosis(annualise(acwi[:,60:72],12)))

"mean"

0.0630025155804051

"std"

0.17698557151531455

"skewness"

-0.744092532275397

"kurtosis"

0.7989998205031514

In [49]:
ACWI_period_returns = returns_usd[:ACWI_Net] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(ACWI_period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1562,1.339,0.0595,0.3651,-2.65,-0.9712,-0.2175,0.122,2.553,0.381,0.1197


In [38]:
M = [1 2 1; 3 4 1; 5 6 1]
cov(M')

3×3 Matrix{Float64}:
 0.333333  0.666667  1.0
 0.666667  2.33333   4.0
 1.0       4.0       7.0

In [39]:
M2 = [1 2 1; 3 4 1; 5 6 missing]
cov(M2')

MethodError: MethodError: Cannot `convert` an object of type Missing to an object of type Float64

Closest candidates are:
  convert(::Type{Float64}, !Matched::Measures.AbsoluteLength)
   @ Measures C:\Users\matsz\.julia\packages\Measures\PKOxJ\src\length.jl:12
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex}
   @ DualNumbers C:\Users\matsz\.julia\packages\DualNumbers\5knFX\src\dual.jl:24
  convert(::Type{T}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S}
   @ Ratios C:\Users\matsz\.julia\packages\Ratios\FsiCW\src\Ratios.jl:51
  ...
